# Building a Collaborative Filtering news recommender
Work in progress

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch
import tensorflow as tf

In [2]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")

## Manual pre-processing of data

In [3]:
raw_behaviour = pd.read_csv(
    "MIND/MINDsmall_train/behaviors.tsv", 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset consist of {len(raw_behaviour)} number of interactions.")
raw_behaviour.head()

The dataset consist of 156965 number of interactions.


,impressionId,userId,timestamp,click_history,impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [4]:
## Indexize users
unique_userIds = raw_behaviour['userId'].unique()
# Allocate a unique index for each user, but let the zeroth index be a UNK index:
ind2user = {idx +1: itemid for idx, itemid in enumerate(unique_userIds)}
user2ind = {itemid : idx for idx, itemid in ind2user.items()}
print(f"We have {len(user2ind)} unique users in the dataset")

# Create a new column with userIdx:
raw_behaviour['userIdx'] = raw_behaviour['userId'].map(lambda x: user2ind.get(x,0))
raw_behaviour.head(5)

We have 50000 unique users in the dataset


,impressionId,userId,timestamp,click_history,impressions,userIdx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5


In [33]:
len (raw_behaviour.userIdx.unique())


50000

## Load article data


In [5]:
news = pd.read_csv(
    "MIND/MINDsmall_train/news.tsv", 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
news.head(2)
# Build index of items
ind2item = {idx +1: itemid for idx, itemid in enumerate(news['itemId'].values)}
item2ind = {itemid : idx for idx, itemid in ind2item.items()}

news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [10]:
# Indexize click history field
def process_click_history(s):
    list_of_strings = str(s).split(" ")
    return [item2ind.get(l, 0) for l in list_of_strings]
        
raw_behaviour['click_history_idx'] = raw_behaviour.click_history.map(lambda s:  process_click_history(s))
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,noclicks,click,epochhrs,noclick,click_history_idx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,[50689],33900,437073.0,50689,"[6893, 10050, 15556, 21467, 26358, 4946, 14071..."
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187,437106.0,37405,"[25816, 2334, 8524, 12087, 13463, 14202, 12733..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767,437143.0,39528,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[50689, 50106, 50022]",50715,437069.0,50689,"[13827, 19085, 28506, 7024, 22910, 16667, 1559..."
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475,437104.0,2006,"[23643, 4853, 27686, 31189]"


In [11]:
# collect one click and one no-click from impressions:
def process_impression(s):
    list_of_strings = s.split(" ")
    itemid_rel_tuple = [l.split("-") for l in list_of_strings]
    noclicks = []
    for entry in itemid_rel_tuple:
        if entry[1] =='0':
            noclicks.append(entry[0])
        if entry[1] =='1':
            click = entry[0]
    return noclicks, click

raw_behaviour['noclicks'], raw_behaviour['click'] = zip(*raw_behaviour['impressions'].map(process_impression))
# We can then indexize these two new columns:
raw_behaviour['noclicks'] = raw_behaviour['noclicks'].map(lambda list_of_strings: [item2ind.get(l, 0) for l in list_of_strings])
raw_behaviour['click'] = raw_behaviour['click'].map(lambda x: item2ind.get(x,0))

In [12]:
raw_behaviour.head()

,impressionId,userId,timestamp,click_history,impressions,userIdx,noclicks,click,epochhrs,noclick,click_history_idx
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,1,[50689],33900,437073.0,50689,"[6893, 10050, 15556, 21467, 26358, 4946, 14071..."
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,2,"[37405, 41306, 34907, 35307, 44370, 37210, 439...",32187,437106.0,37405,"[25816, 2334, 8524, 12087, 13463, 14202, 12733..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,3,"[39528, 33356, 38720, 43459, 794, 38061, 39830...",5767,437143.0,39528,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,4,"[50689, 50106, 50022]",50715,437069.0,50689,"[13827, 19085, 28506, 7024, 22910, 16667, 1559..."
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,5,"[2006, 33272, 39220, 37210, 45683, 50113, 3663...",31475,437104.0,2006,"[23643, 4853, 27686, 31189]"


In [13]:
# convert timestamp value to hours since epoch
raw_behaviour['epochhrs'] = pd.to_datetime(raw_behaviour['timestamp']).values.astype(np.int64)/(1e6)/1000/3600
raw_behaviour['epochhrs'] = raw_behaviour['epochhrs'].round()
raw_behaviour[['click','epochhrs']].groupby("click").min("epochhrs").reset_index()

,click,epochhrs
0,13,437047.0
1,28,437100.0
2,48,437079.0
3,75,437069.0
4,86,437150.0
...,...,...
6346,51263,437079.0
6347,51269,437063.0
6348,51271,437069.0
6349,51275,437063.0


# Modeling



In [14]:
raw_behaviour['noclick'] = raw_behaviour['noclicks'].map(lambda x : x[0])
behaviour = raw_behaviour[['epochhrs','userIdx','click_history_idx','noclick','click']]
behaviour.head()

,epochhrs,userIdx,click_history_idx,noclick,click
0,437073.0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",50689,33900
1,437106.0,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...",37405,32187
2,437143.0,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...",39528,5767
3,437069.0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...",50689,50715
4,437104.0,5,"[23643, 4853, 27686, 31189]",2006,31475


In [35]:
np.sort(behaviour.groupby('userIdx')['click'].count(), o)

array([ 1,  1,  1, ..., 41, 44, 62])

In [15]:
# Let us use the last 10pct of the data as our validation data:
test_time_th = behaviour['epochhrs'].quantile(0.9)
train = behaviour[behaviour['epochhrs']< test_time_th]
valid =  behaviour[behaviour['epochhrs']>= test_time_th]

In [16]:
train

,epochhrs,userIdx,click_history_idx,noclick,click
0,437073.0,1,"[6893, 10050, 15556, 21467, 26358, 4946, 14071...",50689,33900
1,437106.0,2,"[25816, 2334, 8524, 12087, 13463, 14202, 12733...",37405,32187
2,437143.0,3,"[5477, 4207, 11684, 7704, 8124, 23394, 22970, ...",39528,5767
3,437069.0,4,"[13827, 19085, 28506, 7024, 22910, 16667, 1559...",50689,50715
4,437104.0,5,"[23643, 4853, 27686, 31189]",2006,31475
...,...,...,...,...,...
156956,437049.0,15553,"[20696, 5477, 8307, 5600, 4292, 11871, 23832, ...",38518,36569
156957,437083.0,12037,"[23149, 20177, 5662, 21671, 11743, 12143, 2271...",13360,43109
156958,437105.0,8333,"[3176, 2942, 4211, 195, 22307, 30118, 22254, 1...",36915,32187
156961,437119.0,1036,"[27148, 6338, 19239, 6951, 7631, 20288, 8014, ...",39247,45322


In [17]:
class MindDataset(Dataset):
    # A fairly simple torch dataset module that can take a pandas dataframe (as above), 
    # and convert the relevant fields into a dictionary of arrays that can be used in a dataloader
    def __init__(self, df):
        # Create a dictionary of tensors out of the dataframe
        self.data = {
            'userIdx' : torch.tensor(df.userIdx.values),
            'click' : torch.tensor(df.click.values),
            'noclick' : torch.tensor(df.noclick.values)
        }
    def __len__(self):
        return len(self.data['userIdx'])
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.data.items()}

In [18]:
# Build datasets and dataloaders of train and validation dataframes:
bs = 1024
ds_train = MindDataset(train)
train_loader = DataLoader(ds_train, batch_size=bs, shuffle=True)
ds_valid = MindDataset(valid)
valid_loader = DataLoader(ds_valid, batch_size=bs, shuffle=False)

batch = next(iter(train_loader))

In [22]:
valid

,epochhrs,userIdx,click_history_idx,noclick,click
13,437148.0,14,"[2382, 15533, 24759, 7248, 4886, 15003, 8705, ...",4168,39967
20,437147.0,21,"[6057, 17604, 10612, 28808, 24196, 14578, 2434...",49209,35691
39,437150.0,40,"[1823, 12143, 11871, 10865, 20212, 12896, 1258...",42661,32961
49,437149.0,50,"[4106, 17964, 3682, 31494, 31787, 31953, 8645,...",28568,32931
58,437147.0,59,"[5058, 19980, 4292, 14669, 16758, 20739, 25466...",35691,34170
...,...,...,...,...,...
156947,437148.0,4554,"[22289, 11026, 6851, 14472, 24265, 1166, 7691,...",31290,22968
156959,437152.0,50000,"[22941, 25895, 29202]",35105,49840
156960,437158.0,7282,"[21178, 19891, 2656, 14202, 16230, 21717, 1626...",36709,39852
156962,437147.0,4834,"[19161, 26886, 30517, 27148, 22579, 9315, 1336...",37872,40345


In [19]:
batch["noclick"]

tensor([38493, 31542, 50689,  ..., 43608, 36333, 33378])

## Model



In [23]:
# Build a matrix factorization model
class NewsMF(pl.LightningModule):
    def __init__(self, num_users, num_items, dim = 10):
        super().__init__()
        self.dim=dim
        self.useremb = nn.Embedding(num_embeddings=num_users, embedding_dim=dim)
        self.itememb = nn.Embedding(num_embeddings=num_items, embedding_dim=dim)
    
    def forward(self, user, item):
        batch_size = user.size(0)
        uservec = self.useremb(user)
        itemvec = self.itememb(item)

        score = (uservec*itemvec).sum(-1).unsqueeze(-1)
        
        return score
    
    def training_step(self, batch, batch_idx):
        batch_size = batch['userIdx'].size(0)

        score_click = self.forward(batch['userIdx'], batch['click'])
        score_noclick = self.forward(batch['userIdx'], batch['noclick'])
        
        scores_all = torch.concat((score_click, score_noclick), dim=1)
        # Compute loss as cross entropy (categorical distribution between the clicked and the no clicked item)
        loss = F.cross_entropy(input=scores_all, target=torch.zeros(batch_size, device=scores_all.device).long())
        return loss
    
    def validation_step(self, batch, batch_idx):
        # for now, just do the same computation as during training
        loss = self.training_step(batch, batch_idx)
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
mf_model = NewsMF(num_users=len(ind2user)+1, num_items = len(ind2item)+1)

In [24]:
trainer = pl.Trainer(max_epochs=10, accelerator="gpu",logger=logger)
trainer.fit(model=mf_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tb_logs/my_model

  | Name    | Type      | Params
--------------------------------------
0 | useremb | Embedding | 500 K 
1 | itememb | Embedding | 512 K 
--------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.051     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/mslokom/Documents/RecSys_News/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/mslokom/Documents/RecSys_News/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 138/138 [00:02<00:00, 50.79it/s, v_num=0, val_loss=1.660]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 138/138 [00:02<00:00, 50.49it/s, v_num=0, val_loss=1.660]


## Sense check

In [25]:
USER_ID = 2350

**Suggested Items**

In [26]:
# Create item_ids and user ids list
item_id = list(ind2item.keys())
userIdx =  [USER_ID]*len(item_id)

preditions = mf_model.forward(torch.IntTensor(userIdx), torch.IntTensor(item_id))

# Select top 10 argmax
top_index = torch.topk(preditions.flatten(), 10).indices

# Filter for top 10 suggested items
filters = [ind2item[ix.item()] for ix in top_index]
news[news["itemId"].isin(filters)]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
5905,N10814,finance,markets,US considers dropping some tariffs on China,According to five people briefed on the discus...,https://assets.msn.com/labs/mind/AAJQZHE.html,"[{""Label"": ""China"", ""Type"": ""G"", ""WikidataId"":...","[{""Label"": ""White House"", ""Type"": ""F"", ""Wikida..."
8994,N17634,finance,finance-companies,Oleg Deripaska-Linked Firm Was Raided for Undi...,U.S. prosecutors are seeking records from a co...,https://assets.msn.com/labs/mind/AAIYxpa.html,[],"[{""Label"": ""Oleg Deripaska"", ""Type"": ""P"", ""Wik..."
10050,N2175,lifestyle,lifestylebuzz,Berea police search for two missing 16-year-ol...,"BEREA, Ohio The Berea Police Department is as...",https://assets.msn.com/labs/mind/AAJg9dl.html,"[{""Label"": ""Berea, Ohio"", ""Type"": ""G"", ""Wikida...","[{""Label"": ""Berea, Ohio"", ""Type"": ""G"", ""Wikida..."
10468,N60242,video,lifestyle,Why You Should Never Squat Over the Toilet Seat,You should never squat over the toilet. Watch ...,https://assets.msn.com/labs/mind/AAIDPQy.html,[],[]
13841,N11693,news,newsus,Racist video prompts action at UConn,STORRS - UConn's Undergraduate Student Governm...,https://assets.msn.com/labs/mind/AAJ76Oh.html,"[{""Label"": ""University of Connecticut"", ""Type""...","[{""Label"": ""University of Connecticut"", ""Type""..."
16642,N626,weather,weathertopstories,Fire Crews Knock Down Brush Fire Near Route 24...,Firefighters have extinguished a brush fire th...,https://assets.msn.com/labs/mind/AAJyfsa.html,[],"[{""Label"": ""California State Route 242"", ""Type..."
27341,N21457,sports,football_nfl,Watch: Lions burn Packers with 66-yard flea-fl...,The Lions brought out the classic gadget play ...,https://assets.msn.com/labs/mind/AAIM06u.html,"[{""Label"": ""Detroit Lions"", ""Type"": ""O"", ""Wiki...","[{""Label"": ""Detroit Lions"", ""Type"": ""O"", ""Wiki..."
41910,N21204,weather,weathertopstories,Firefighters Battle Bushfires North of Sydney ...,Volunteer firefighters with the Ingleside Rura...,https://assets.msn.com/labs/mind/BBWBM1k.html,"[{""Label"": ""Sydney"", ""Type"": ""G"", ""WikidataId""...","[{""Label"": ""New South Wales Rural Fire Service..."
42484,N3449,news,newscrime,Woman accused of sneaking into Bronx Zoo lion ...,The woman who allegedly snuck into a lion exhi...,https://assets.msn.com/labs/mind/BBWtpQB.html,"[{""Label"": ""Bronx Zoo"", ""Type"": ""F"", ""Wikidata...","[{""Label"": ""Bronx Zoo"", ""Type"": ""F"", ""Wikidata..."
46894,N36177,news,newsus,Miami Soccer Stadium Project Kicked To Decembe...,NaN,https://assets.msn.com/labs/mind/BBWFdRw.html,"[{""Label"": ""Fort St. Angelo"", ""Type"": ""F"", ""Wi...",[]


**Historical items**

In [29]:
click_ids = behaviour[behaviour["userIdx"]==USER_ID]["click"].values
ll = lambda x: "N"+str(x)

click_ids = [ll(each) for each in click_ids]

news[news["itemId"].isin(click_ids)]

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
20810,N33900,sports,football_nfl,Patriots Vs. Jets Live: New England Steamrolls...,Final Patriots - : The New England Patriots im...,https://assets.msn.com/labs/mind/AAJ7aqU.html,"[{""Label"": ""New England Patriots"", ""Type"": ""O""...","[{""Label"": ""New England Patriots"", ""Type"": ""O""..."
28754,N32675,news,newsus,Northbound Texas 130 in Pflugerville closing f...,Northbound Texas 130 at Pflugerville Parkway w...,https://assets.msn.com/labs/mind/AAJpxWP.html,"[{""Label"": ""Texas"", ""Type"": ""G"", ""WikidataId"":...","[{""Label"": ""Texas"", ""Type"": ""G"", ""WikidataId"":..."
33795,N34112,travel,travelarticle,Budweiser Clydesdale to strut stuff at Jackson...,Residents can get an up-close glimpse or selfi...,https://assets.msn.com/labs/mind/BBWLjXu.html,"[{""Label"": ""Winn-Dixie"", ""Type"": ""O"", ""Wikidat...","[{""Label"": ""Winn-Dixie"", ""Type"": ""O"", ""Wikidat..."
33846,N32603,news,newsus,Joke Made About President Trump During North S...,An inappropriate joke made during a North Surr...,https://assets.msn.com/labs/mind/BBWJlkC.html,[],"[{""Label"": ""White House"", ""Type"": ""F"", ""Wikida..."
43516,N39247,news,newsus,Allegheny County Controller Chelsa Wagner Goes...,Allegheny County Controller Chelsa Wagner goes...,https://assets.msn.com/labs/mind/BBWBTtb.html,[],"[{""Label"": ""Detroit"", ""Type"": ""G"", ""WikidataId..."
44747,N45975,video,news,Desperation? Trump's top aide now suing Trump ...,The potential linchpin to the impeachment case...,https://assets.msn.com/labs/mind/BBWBZ1g.html,[],"[{""Label"": ""Ari Melber"", ""Type"": ""P"", ""Wikidat..."
45916,N43459,sports,football_nfl,Will Cain Ranks Three Teams Ahead Of Patriots ...,https://www.dailymotion.com/embed/video/x ns u...,https://assets.msn.com/labs/mind/BBWwsaG.html,"[{""Label"": ""New England Patriots"", ""Type"": ""O""...","[{""Label"": ""New England Patriots"", ""Type"": ""O""..."


In [31]:
click_ids

['N31222',
 'N43459',
 'N50715',
 'N34112',
 'N39247',
 'N32603',
 'N45975',
 'N32675',
 'N33900']

In [30]:
# Load the extension and start TensorBoard

%load_ext tensorboard
%tensorboard --logdir logs
